# Polar Coding Notebook

In [ ]:
import numpy as np
from numpy.random import randint
import matplotlib.pyplot as plt

Helper Functions

In [ ]:
def polar_transform(u):
    N = len(u)
    if N == 1:
        return u.copy()

    u_even = u[0:N:2]
    u_odd = u[1:N:2]

    x_upper = polar_transform((u_even ^ u_odd) % 2)
    x_lower = polar_transform(u_odd)
    return np.concatenate([x_upper, x_lower])


def bhattacharyya_parameter(p, N):
    Z = np.array([2 * np.sqrt(p * (1 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append((2 * z) - (z ** 2))  # Upper channel
            Z_next.append(z ** 2)          # Lower channel
        Z = np.array(Z_next)
    return Z


def select_frozen_bits(N, K, p):
    Z = bhattacharyya_parameter(p, N)
    indices = np.argsort(Z)
    info_index = indices[:K]
    frozen = np.ones(N, dtype=bool)
    frozen[info_index] = False
    return frozen, info_index


def successive_cancellation_decode(llr, frozen, u_frozen):
    # llr is the log-likelihood ratio input
    N = len(llr)
    u_hat = np.zeros(N, dtype=np.int8)

    def sc_decode_recursive(llr, depth=0):
        n = len(llr)
        if n == 1:
            return np.array([0 if llr[0] >= 0 else 1], dtype=np.int8)
        
        n2 = n//2
        llr_upper = np.sign(llr[:n2]*llr[n2:]) * np.minimum(np.abs(llr[:n2]), np.abs(llr[n2:]))
        u_hat_upper = sc_decode_recursive(llr_upper, depth+1)

        llr_lower = llr[n2:] + ((-1) ** u_hat_upper) * llr[:n2]
        u_hat_lower = sc_decode_recursive(llr_lower, depth+1)

        return np.concatenate([u_hat_upper ^ u_hat_lower, u_hat_lower])
    
    u_hat = sc_decode_recursive(llr)
    for i in range(N):
        if frozen[i]:
            u_hat[i] = u_frozen[i]
    return u_hat

Channels

In [ ]:
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return x ^ flips


def bsc_llr(y, p):
    alpha = np.log((1 -p) / p)
    return (1 - (2 * y)) * alpha

Simulations